In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt

from mlxtend.plotting import plot_confusion_matrix

from colorama import Fore, Back, Style 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier


import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [3]:
heart_data = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
heart_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
z = heart_data["DEATH_EVENT"]
x = heart_data.drop(["DEATH_EVENT","age","anaemia","creatinine_phosphokinase","diabetes", "high_blood_pressure", "serum_sodium", "sex", "smoking"],axis=1)
y = z.copy()
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=2)

In [5]:
x_train.head(20)

,ejection_fraction,platelets,serum_creatinine,time
53,50,351000.00,1.00,44
172,60,176000.00,1.10,146
23,60,368000.00,0.80,22
98,25,318000.00,1.20,85
244,38,264000.00,1.80,213
120,60,210000.00,1.50,95
153,35,263358.03,1.18,119
192,40,260000.00,0.70,175
225,60,265000.00,1.40,205
201,60,377000.00,1.00,186


In [6]:
y_train.head(20)

53     1
172    0
23     0
98     0
244    0
120    0
153    0
192    0
225    0
201    0
54     1
4      1
151    0
249    0
163    1
144    1
209    0
227    0
24     1
278    0
Name: DEATH_EVENT, dtype: int64

In [7]:
X = x_train.copy().values
y = y_train.copy().values

le = LabelEncoder()

y = le.fit_transform(y)
y = to_categorical(y)

In [8]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow import keras

model = Sequential()

model.add(Dense(64, activation='relu', input_shape=[4]))
model.add(Dense(64))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer=keras.optimizers.Adam(lr=0.01), loss='binary_crossentropy',
              metrics=['acc'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 4,610
Trainable params: 4,610
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X,y, epochs=200)

Epoch 1/200
8/8 [==============================] - 1s 3ms/step - loss: 23399.9462 - acc: 0.5172
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 5675.9325 - acc: 0.5565
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 4472.3170 - acc: 0.4519
Epoch 4/200
8/8 [==============================] - 0s 3ms/step - loss: 3878.3779 - acc: 0.5641
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 2075.7224 - acc: 0.6974
Epoch 6/200
8/8 [==============================] - 0s 3ms/step - loss: 2686.5598 - acc: 0.4689
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 2127.7824 - acc: 0.6925
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 2765.0489 - acc: 0.4691
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 1102.7265 - acc: 0.4958
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1070.1892 - acc: 0.7057
Epoch 11/200
8/8 [==============================

In [10]:
# gradientboost_clf = GradientBoostingClassifier(max_depth=2, random_state=1)
# gradientboost_clf.fit(x_train,y_train)
# gradientboost_pred = gradientboost_clf.predict(x_test)
# gradientboost_acc = accuracy_score(y_test, gradientboost_pred)

# accuracy_list = []
# accuracy_list.append(100*gradientboost_acc)

In [11]:
# print("Accuracy of GradientBoostingClassifier is : ", 100*gradientboost_acc)

In [12]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)

In [13]:
tfmodel = converter.convert()

In [14]:
open('heart_disease.tflite', 'wb').write(tfmodel)

20168